# Getting Started with Hyperwave

This notebook demonstrates how to use the Hyperwave community package to run GPU-accelerated FDTD photonics simulations.

**What you'll learn:**
1. How to configure your API key
2. How to estimate simulation costs
3. How to build a photonic structure from a gdsfactory component
4. How to run a simulation and visualize results
5. How to use early stopping for faster convergence

**Prerequisites:**
- A Hyperwave API key (get one free at [spinsphotonics.com](https://spinsphotonics.com))
- Python 3.9+

## 1. Installation

Install the hyperwave-community package:

In [1]:
# Install from GitHub
!pip install git+https://github.com/spinsphotonics/hyperwave-community.git

# Or install locally if you have the repo cloned:
# !pip install -e /path/to/hyperwave-community

  Cloning https://github.com/spinsphotonics/hyperwave-community.git to /private/var/folders/wq/qgpnwsdx6nv6qtzczxy0mnjc0000gn/T/pip-req-build-48w65y9a
  Running command git clone --filter=blob:none --quiet https://github.com/spinsphotonics/hyperwave-community.git /private/var/folders/wq/qgpnwsdx6nv6qtzczxy0mnjc0000gn/T/pip-req-build-48w65y9a
  Resolved https://github.com/spinsphotonics/hyperwave-community.git to commit e39f8bef5e426e09f4d58e73e2f04cadeae011d0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## 2. Configure Your API Key

You can set your API key in two ways:
1. Environment variable: `export HYPERWAVE_API_KEY=your-key`
2. In code using `configure_api()`

In [ ]:
import hyperwave_community as hwc

# Option 1: Set your API key directly
API_KEY = "your-api-key-here"  # Replace with your actual key

# Option 2: Or use environment variable
# import os
# os.environ['HYPERWAVE_API_KEY'] = 'your-api-key-here'

# Configure and verify API key
hwc.configure_api(api_key=API_KEY)

# Check account info (verifies API key and shows credit balance)
account = hwc.get_account_info()
if account:
    print(f"✓ API key valid")
    print(f"  Credits: {account['credits_balance']:.4f} (${account['credits_balance_usd']:.2f})")
else:
    print("✗ API key invalid or not configured")

## 3. Estimate Simulation Cost (Free)

Before running a simulation, you can estimate the cost. This endpoint is free and doesn't require authentication.

In [3]:
# Estimate cost for a medium-sized simulation
estimate = hwc.estimate_cost(
    structure_shape=(3, 3600, 500, 140),  # (3, Lx, Ly, Lz)
    max_steps=20000,
    gpu_type="B200"
)

print(f"Estimated time: {estimate['estimated_seconds']:.1f} seconds")
print(f"Estimated cost: {estimate['estimated_credits']:.4f} credits")
print(f"Note: {estimate['note']}")

Error estimating cost: 404 Client Error: Not Found for url: https://hyperwave-cloud.onrender.com/estimate_cost


TypeError: 'NoneType' object is not subscriptable

## 4. Quick Start: All-in-One Simulation Setup

The easiest way to get started is using `prepare_simulation_inputs()`. This function:
1. Builds a structure recipe from a gdsfactory component
2. Creates monitors at all ports
3. Solves for the waveguide mode source

All in one API call! This runs on CPU and **does not consume credits**.

In [ ]:
# Prepare all inputs for an MMI 1x2 splitter simulation
inputs = hwc.prepare_simulation_inputs(
    component_name="mmi1x2",           # gdsfactory component name
    resolution_nm=30,                   # 30nm grid resolution
    wl_min_um=1.55,                     # wavelength: 1550nm
    wl_max_um=1.55,
    n_freqs=1,                          # single frequency
    source_port="o1",                   # inject from port o1
    api_key=API_KEY
)

print(f"Structure dimensions: {inputs['dimensions']}")
print(f"Source field shape: {inputs['source_field'].shape}")
print(f"Frequency band: {inputs['freq_band']}")
print(f"Monitors: {[m['name'] for m in inputs['monitors']]}")
print(f"Build time: {inputs['metadata']['build_time_s']:.1f}s")

## 5. Run the Simulation

Now we can run the FDTD simulation on a GPU. This is where credits are consumed.

In [ ]:
# Estimate cost before running
Lx, Ly, Lz = inputs['dimensions']
estimate = hwc.estimate_cost(
    grid_points=Lx * Ly * Lz,
    max_steps=10000,
    gpu_type="H100"
)
print(f"Grid points: {Lx * Ly * Lz:,}")
print(f"Estimated cost: {estimate['estimated_credits']:.4f} credits")
print(f"Estimated time: {estimate['estimated_seconds']:.0f} seconds")

In [ ]:
# Ready to simulate! Uncomment below to run (will consume credits)

In [ ]:
# UNCOMMENT TO RUN THE SIMULATION (will consume credits)

# results = hwc.simulate_from_recipe(
#     structure_recipe=inputs['structure_recipe'],
#     source_field=inputs['source_field'],
#     source_offset=inputs['source_offset'],
#     freq_band=inputs['freq_band'],
#     monitors=inputs['monitors'],
#     max_steps=10000,
#     check_every_n=1000,
#     gpu_type="H100",
#     **inputs['absorber_params'],
#     api_key=API_KEY
# )
#
# print(f"Simulation completed in {results['sim_time']:.2f}s")
# print(f"Performance: {results['performance']:,.0f} grid-points x steps/s")
# print(f"GPU: {results['gpu_type']}")

## 6. Alternative: Step-by-Step Workflow

If you need more control, you can call each step separately. All these CPU functions are **free** (no credits).

In [ ]:
# Step 1: Build the structure recipe
recipe_result = hwc.build_recipe(
    component_name="mmi2x2",
    resolution_nm=30,
    n_core=3.48,       # Silicon
    n_clad=1.44,       # SiO2
    wg_height_um=0.22, # 220nm waveguide
    api_key=API_KEY
)

print(f"Structure dimensions: {recipe_result['dimensions']}")
print(f"Ports: {list(recipe_result['port_info'].keys())}")

In [ ]:
# Step 2: Build monitors from ports
monitor_result = hwc.build_monitors(
    port_info=recipe_result['port_info'],
    dimensions=recipe_result['dimensions'],
    source_port="o1",  # which port to inject light
    api_key=API_KEY
)

print(f"Monitors: {list(monitor_result['monitor_names'].keys())}")
print(f"Source port: {monitor_result['source_port_name']}")
print(f"Source position: x={monitor_result['source_position']}")

In [ ]:
# Step 3: Compute frequency band
freq_result = hwc.compute_freq_band(
    wl_min_um=1.5,   # 1500nm
    wl_max_um=1.6,   # 1600nm
    n_freqs=5,       # 5 wavelength points
    resolution_um=0.03,
    api_key=API_KEY
)

print(f"Frequency band: {freq_result['freq_band']}")
print(f"Wavelengths: {freq_result['wavelengths_um']}")

In [ ]:
# Step 4: Solve for waveguide mode source
source_result = hwc.solve_mode_source(
    density_core=recipe_result['density_core'],
    density_clad=recipe_result['density_clad'],
    source_x_position=monitor_result['source_position'],
    mode_bounds=monitor_result['mode_bounds'],
    layer_config=recipe_result['layer_config'],
    eps_values=recipe_result['eps_values'],
    freq_band=freq_result['freq_band'],
    mode_num=0,  # fundamental mode
    api_key=API_KEY
)

print(f"Source field shape: {source_result['source_field'].shape}")
print(f"Source offset: {source_result['source_offset']}")

In [ ]:
# Step 5: Get default absorber parameters
absorber = hwc.get_default_absorber_params(
    structure_dimensions=recipe_result['dimensions'],
    api_key=API_KEY
)

print(f"Absorption widths: {absorber['absorption_widths']}")
print(f"Absorption coefficient: {absorber['absorption_coeff']}")

## 7. Early Stopping Simulation

For faster convergence, use `early_stopping_simulate()`. This monitors power at all ports and stops when the simulation has converged, potentially saving significant compute time.

In [ ]:
# Early stopping is useful for:
# - Long simulations where you're not sure how many steps are needed
# - Structures that converge quickly
# - Reducing costs on well-behaved simulations

print("Early stopping parameters:")
print("  max_steps: 200000 (upper limit)")
print("  relative_threshold: 0.01 (1% power change)")
print("  min_stable_checks: 3 (must be stable for 3 checks)")
print("")
print("The simulation will stop early once power stabilizes at all output ports.")

## 8. Available GPU Types

Choose the GPU that fits your budget and performance needs:

In [ ]:
gpu_info = {
    'B200': {'performance': '25B pts/s', 'multiplier': '2.5x', 'best_for': 'Largest simulations'},
    'H200': {'performance': '12B pts/s', 'multiplier': '2.0x', 'best_for': 'Very large simulations'},
    'H100': {'performance': '5.3B pts/s', 'multiplier': '1.5x', 'best_for': 'Large simulations (recommended)'},
    'A100-80GB': {'performance': '4.1B pts/s', 'multiplier': '1.0x', 'best_for': 'Memory-intensive'},
    'A100-40GB': {'performance': '3.5B pts/s', 'multiplier': '0.8x', 'best_for': 'Good balance'},
    'L40S': {'performance': '2.5B pts/s', 'multiplier': '0.7x', 'best_for': 'Cost-effective'},
    'A10G': {'performance': '1.5B pts/s', 'multiplier': '0.4x', 'best_for': 'Small simulations'},
    'T4': {'performance': '800M pts/s', 'multiplier': '0.3x', 'best_for': 'Testing/prototyping'},
}

print("GPU Options:")
print("-" * 70)
for gpu, info in gpu_info.items():
    print(f"{gpu:12} | {info['performance']:12} | {info['multiplier']:5} cost | {info['best_for']}")

## 9. Supported gdsfactory Components

You can use any component from gdsfactory. Common photonic components include:

In [ ]:
components = [
    "straight",           # Simple straight waveguide
    "bend_euler",         # Euler bend
    "bend_circular",      # Circular bend  
    "mmi1x2",             # 1x2 MMI splitter
    "mmi2x2",             # 2x2 MMI coupler
    "coupler",            # Directional coupler
    "coupler_ring",       # Ring coupler
    "taper",              # Linear taper
    "crossing",           # Waveguide crossing
]

print("Common components:")
for comp in components:
    print(f"  - {comp}")

print("\nSee gdsfactory docs for full list: https://gdsfactory.github.io/gdsfactory/")

## 10. Next Steps

- **Get your API key**: [spinsphotonics.com](https://spinsphotonics.com)
- **Documentation**: [docs.spinsphotonics.com](https://docs.spinsphotonics.com)
- **Examples**: Check out more examples in this folder
- **Support**: [support@spinsphotonics.com](mailto:support@spinsphotonics.com)